In [1]:
#Import dependencies
import pandas as pd
import os

In [2]:
#Import csv and create dataframe
purchase_data_csv = os.path.join('Resources/purchase_data.csv')
purchase_data_df = pd.read_csv(purchase_data_csv)

purchase_data_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
#Player Count
#Display the total number of players
total_players = pd.Series(len(purchase_data_df['SN'].unique()))
total_players_df = pd.DataFrame({
    'Total Players': total_players
})

total_players_df

,Total Players
0,576


In [4]:
#Purchasing Analysis (Total)
#Run basic calculations to obtain number of unique items, average price, etc.
no_unique_items = pd.Series(purchase_data_df['Item Name'].nunique())
average_price = pd.Series(purchase_data_df['Price'].mean())
no_purchases = pd.Series(purchase_data_df['Purchase ID'].count())
total_revenue = pd.Series(purchase_data_df['Price'].sum())

#Summary dataframe with cleaner formatting
purchasing_analysis_df = pd.DataFrame({
    'Number of Unique Items': no_unique_items,
    'Average Price': average_price,
    'Number of Purchases': no_purchases,
    'Total Revenue': total_revenue
})
purchasing_analysis_df['Average Price'] = purchasing_analysis_df['Average Price'].map('${:.2f}'.format)
purchasing_analysis_df['Total Revenue'] = purchasing_analysis_df['Total Revenue'].map('${:.2f}'.format)

#Display summary dataframe
purchasing_analysis_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


In [5]:
#Gender Demographics
#Percentage and Count of Male Players, Female Players & Other/Non-Disclosed
gendered_df = purchase_data_df.drop_duplicates(subset=['SN'])
genders = gendered_df['Gender'].value_counts()
genders_percent = (genders / total_players[0])

genders_percent_df = pd.DataFrame({
    'Total Count': genders,
    'Percentage of Players': genders_percent
})
genders_percent_df['Percentage of Players'] = genders_percent_df['Percentage of Players'].map('{:.2%}'.format)

genders_percent_df

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [6]:
#Purchasing Analyis (Gender)
#Run basic calculations to obtain purchase count
gendered_purchase_df = purchase_data_df.groupby('Gender')
gendered_purchase_counts = gendered_purchase_df['Gender'].count()

#Average purchase price
gendered_purchase_avg = gendered_purchase_df['Price'].mean()
gendered_purchase_avg = gendered_purchase_avg.map('${:.2f}'.format)

#Total Purchase Value - .map(format) here kept throwing an error
gendered_purchase_total = gendered_purchase_df['Price'].sum()
#gendered_purchase_total = gendered_purchase_total.map('${:.2f}'.format)

#Average purchase total per person by gender
gendered_purchase_avg_per = (gendered_purchase_total / genders)
gendered_purchase_avg_per = gendered_purchase_avg_per.map('${:.2f}'.format)

#Display summary dataframe - w/ .map(format) workaround
gendered_purchase_table_df = pd.DataFrame({
    'Purchase Count': gendered_purchase_counts,
    'Average Purchase Price': gendered_purchase_avg,
    'Total Purchase Value': gendered_purchase_total.map('${:.2f}'.format),
    'Average Total Purchase per Person': gendered_purchase_avg_per
})
gendered_purchase_table_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [7]:
#Age Demographics
#Establish bins for ages
bins = [0, 9, 14, 19, 24, 29, 34, 39, 50]
bin_groups = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

#Categorize the existing players using the age bins. Hint: use pd.cut()
purchase_data_df['Age Group'] = pd.cut(purchase_data_df['Age'], bins, labels=bin_groups)

#Calculate the numbers and percentages by age group
player_age_groups = purchase_data_df.groupby('Age Group')['SN'].nunique()
player_age_percentage = (player_age_groups / purchase_data_df['SN'].nunique())
#Optional: round the percentage column to two decimal points
player_age_percentage = player_age_percentage.map('{:.2%}'.format)

#Create a summary data frame to hold the results
aged_df = pd.DataFrame({
    'Total Count': player_age_groups,
    'Percentage of Players': player_age_percentage
})

#Display Age Demographics Table
aged_df

,Total Count,Percentage of Players
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [8]:
#Purchasing Analysis (Age)
#Bin the purchase_data dataframe by age
age_purchase_count = purchase_data_df[['Age Group', 'Purchase ID']]
age_purchase_value = purchase_data_df[['Age Group', 'Price']]
unique_sn_per_age = purchase_data_df[['Age Group', 'SN']]

#Run basic calculations to obtain purchase count
age_group_purchase_totals = age_purchase_count.groupby('Age Group').count()

#total purchase value
age_total_purchase_value = age_purchase_value.groupby('Age Group').sum()

#average purchase price
age_purchase_price = (age_total_purchase_value['Price'] / age_group_purchase_totals['Purchase ID'])

#average total purchase per person 
age_avg_purchase_per = (age_total_purchase_value['Price'] / player_age_groups)

#Create a summary data frame to hold the results
#Optional: give the displayed data cleaner formatting
age_purchase_df = pd.DataFrame({
    'Purchase Count': age_group_purchase_totals['Purchase ID'],
    'Average Purchase Price': age_purchase_price.map('${:.2f}'.format),
    'Total Purchase Value': age_total_purchase_value['Price'].map('${:.2f}'.format),
    'Average Total Purchase per Person': age_avg_purchase_per.map('${:.2f}'.format)
})


#Display the summary data frame
age_purchase_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [9]:
#Top Spenders
#Run basic calculations to obtain the results in the table below
top_spenders = purchase_data_df[['Purchase ID', 'SN', 'Price']]
ts_purchase_counts = top_spenders['SN'].value_counts()
ts_avg_price = top_spenders['Price'].groupby(top_spenders['SN']).mean()
ts_total_value = top_spenders['Price'].groupby(top_spenders['SN']).sum()

#Create a summary data frame to hold the results - .map(format) here ends up skewing summary table
top_spenders_df = pd.DataFrame({
    'Purchase Count': ts_purchase_counts,
    'Average Purchase Price': ts_avg_price,
    'Total Purchase Value': ts_total_value
})

#Sort the total purchase value column in descending order
top_spenders_df = top_spenders_df.sort_values('Total Purchase Value', ascending=False)

#Optional: give the displayed data cleaner formatting
top_spenders_df['Average Purchase Price'] = top_spenders_df['Average Purchase Price'].map('${:.2f}'.format)
top_spenders_df['Total Purchase Value'] = top_spenders_df['Total Purchase Value'].map('${:.2f}'.format)

#Display a preview of the summary data frame
top_spenders_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [19]:
#Most Popular Items
#Retrieve the Item ID, Item Name, and Item Price columns
popular_items = purchase_data_df[['Item ID', 'Item Name', 'Price']]

#Group by Item ID and Item Name.
#Perform calculations to obtain purchase count, item price, and total purchase value
item_purchase_count = popular_items.groupby(['Item ID', 'Item Name'])['Price'].count()
item_price = popular_items.groupby(['Item ID', 'Item Name'])['Price'].mean()
item_total_value = popular_items.groupby(['Item ID', 'Item Name'])['Price'].sum()

#Create a summary data frame to hold the results
popular_items_table = pd.DataFrame({
    'Purchase Count': item_purchase_count,
    'Item Price': item_price,
    'Total Purchase Value': item_total_value
})

#Sort the purchase count column in descending order
popular_items_table = popular_items_table.sort_values('Purchase Count', ascending=False)
popular_items_table['Item Price'] = popular_items_table['Item Price'].map('${:.2f}'.format)
popular_items_table['Total Purchase Value'] = popular_items_table['Total Purchase Value'].map('${:.2f}'.format)

#Optional: give the displayed data cleaner formatting


#Display a preview of the summary data frame
popular_items_table.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [24]:
#Most Profitable Items
#Sort the above table by total purchase value in descending order
profitable_items_table = pd.DataFrame({
    'Purchase Count': item_purchase_count,
    'Item Price': item_price,
    'Total Purchase Value': item_total_value
})

#Optional: give the displayed data cleaner formatting
profitable_items_table = profitable_items_table.sort_values('Total Purchase Value', ascending=False)
profitable_items_table['Item Price'] = profitable_items_table['Item Price'].map('${:.2f}'.format)
profitable_items_table['Total Purchase Value'] = profitable_items_table['Total Purchase Value'].map('${:.2f}'.format)

#Display a preview of the data frame
profitable_items_table.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
